## Notebook cào dữ liệu

In [20]:
def write_to_json_file(data, file_name):
  import json
  json_object = json.dumps(data, ensure_ascii=False)
  with open(file_name, "w", encoding="utf-8") as outfile:
    outfile.write(json_object)

In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://www.petmart.vn/hoi-dap-thu-y-mien-phi"
page = requests.get(url, verify=False)
soup = BeautifulSoup(page.text, 'html.parser')

c:\Users\ASUS\AppData\Local\Programs\Python\Python38-32\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.petmart.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
def get_comments(): 
  soup.find(class_='comment')
  comments = soup.find_all("li", class_='comment')
  return comments

def classify_comments(comments):
  ask_list = []
  ask_answer_list = []
  for i in comments:
    content = i.find_all(class_='comment-content')
    if len(content) == 1:
      ask_list.append(content)
    else:
      ask_answer_list.append(content)
  return ask_list, ask_answer_list

Tập dữ liệu được thu thập từ mục hỏi đáp thú y của Petmart có 194 mẫu, câu truyện hỏi đáp. Trong đó chỉ có 59 câu truyện có câu trả lời, còn lại 135 mẫu câu hỏi nhưng bị thiếu câu trả lời.

In [4]:
comments = get_comments() # comments 
[ask_list, ask_answer_list] = classify_comments(comments)
len(comments), len(ask_list), len(ask_answer_list)

(195, 136, 59)

Lấy các mẫu cẩu hỏi và định dạng theo kiểu JSON {ask: "lorem srum...."}
Có 135 mẫu kiểu này

In [5]:
# list only have ask but haven't answer yet
ask_data = []
for item in ask_list:
  ask = (item[0].contents)[0].text
  ask_data.append({"ask": ask})
len(ask_data)

136

Định dạng các câu truyện ngắn theo mẫu câu hỏi - câu trả lời. Mỗi câu truyện sẽ có dạng {"ask": "abc", "answer": "xyz"}.
Đồng thời nếu câu truyện dài thì ta lưu vào list và sẽ có định dạng khác

Hiện tại có 50 câu truyện ngắn chỉ bao gồm có 1 câu hỏi - 1 câu trả lời

In [6]:
ask_answer_data = []
story_list = [] # item with len > 2
for item in ask_answer_list:
  # [ask, answer] = item
  if (len(item) > 2):
    story_list.append(item)
  else:
    [ask, answer] = item
    ask = ask.contents[0].text
    answer = answer.contents[0].text
    ask_answer_data.append({"ask": ask, "answer": answer})
len(ask_answer_data)

50

Câu Chuyện dài sẽ có dạng [{"ask": "hỏi"}, {"answer": "đáp"}, {"ask": "hỏi tiếp"}, {"answer": "đáp tiếp"}]

Hiện tại có 9 câu chuyện dài

In [7]:
stories_data = []
for story in story_list:
  i = 0
  story_data = []
  for item in story:
    if i % 2 == 0:
      ask = item.contents[0].text
      story_data.append({"ask": ask})
    else:
      answer = item.contents[0].text
      story_data.append({"answer": answer})
    i += 1
  stories_data.append(story_data)
print(len(stories_data))

9


Ghi tất cả các dữ liệu đã cào được ra file

In [21]:
write_to_json_file(ask_data, "petmart_only_ask_data.json")
write_to_json_file(ask_answer_data, "petmart_ask_answer_data.json")
write_to_json_file(stories_data, "petmart_stories_data.json")